Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [1]:
!pip install pyspark >> None

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("data_to_csv").getOrCreate()
data = [
("1984", "George Orwell", "Science Fiction", 5000, 1949),
("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)]
df = spark.createDataFrame(data, ['title', 'author', 'genre', 'sales', 'year'])
df.show()

24/06/12 18:30:12 WARN Utils: Your hostname, nic-1-0 resolves to a loopback address: 127.0.1.1; using 192.168.150.159 instead (on interface wlo1)
24/06/12 18:30:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 18:30:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/12 18:30:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/12 18:30:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [4]:
df.repartition(1).write.csv("exampl.csv", header=True)
spark.stop()

— Используя Spark прочитайте данные из файла csv.


In [5]:
spark = SparkSession.builder.appName("csv_to_data").getOrCreate()
df = spark.read.csv("exampl.csv", header=True)
df.show()

24/06/12 18:30:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/12 18:30:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+




— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

In [6]:
sales = df.filter(df.sales > 3000)
sales.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

In [8]:
from pyspark.sql.functions import sum
genre_sales = df.groupBy("genre").agg(sum("sales").alias("total"))
genre_sales.show()

+---------------+------+
|          genre| total|
+---------------+------+
|Southern Gothic|4000.0|
|          Novel|6500.0|
|        Fantasy|3000.0|
|Science Fiction|5000.0|
+---------------+------+



— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [9]:
genre_sales.orderBy(genre_sales.total.desc()).show()

+---------------+------+
|          genre| total|
+---------------+------+
|          Novel|6500.0|
|Science Fiction|5000.0|
|Southern Gothic|4000.0|
|        Fantasy|3000.0|
+---------------+------+



In [10]:
spark.stop()